# GD-12 Transformer Chatbot

### 2023-02-09 (목)

## 라이브러리 버전을 확인

In [47]:
# ! pip uninstall gensim 
# ! pip install --upgrade gensim==3.8.3

In [48]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
import nltk
import gensim
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

import re
import os
import random
import math

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [49]:
print(np.__version__)
print(pd.__version__)
print(tf.__version__)
print(nltk.__version__)
print(gensim.__version__) # pip install --upgrade gensim==3.8.3

1.22.4
1.5.1
2.9.0
3.7
3.8.3


## Step 1. 데이터 다운로드

In [50]:
# /data/ChatbotData.csv
# !wget https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv

In [51]:
import pandas as pd 

df_chatbotdata = pd.read_csv('./data/ChatbotData.csv')
df_chatbotdata.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


## Step 2. 데이터 정제
---
아래 조건을 만족하는 preprocess_sentence() 함수를 구현하세요.

영문자의 경우, 모두 소문자로 변환합니다.  
영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거합니다. 
문장부호 양옆에 공백을 추가하는 등 이전과 다르게 생략된 기능들은 우리가 사용할 토크나이저가 지원하기 때문에 굳이 구현하지 않아도 괜찮습니다!

In [52]:
import re

def to_lower(sentence: str) -> str:
    return sentence.lower()

def add_white_space(sentence: str) -> str:
    return re.sub(r'[" "]+', " ", sentence)

def to_korean_alpha_numeric(sentence: str) -> str:
    return re.sub(r"[^ㄱ-ㅎ가-힣a-zA-Z!.,?0-9]+", " ", sentence)

def strip(sentence: str) -> str:
    return sentence.strip()

def preprocess_sentence(sentence: str) -> str:
    sentence = to_lower(sentence)
    sentence = add_white_space(sentence)
    sentence = to_korean_alpha_numeric(sentence)
    sentence = strip(sentence)
    return sentence

print("슝=3")

슝=3


## Step 3. 데이터 토큰화
---
토큰화에는 KoNLPy의 mecab 클래스를 사용합니다.

아래 조건을 만족하는 build_corpus() 함수를 구현하세요!  

1. 소스 문장 데이터와 타겟 문장 데이터를 입력으로 받습니다.  
2. 데이터를 앞서 정의한 preprocess_sentence() 함수로 정제하고, 토큰화합니다.  
3. 토큰화는 전달받은 토크나이즈 함수를 사용합니다. 이번엔 mecab.morphs 함수를 전달하시면 됩니다.  
4. 토큰의 개수가 일정 길이 이상인 문장은 데이터에서 제외합니다.  
5. 중복되는 문장은 데이터에서 제외합니다. 소스 : 타겟 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사합니다. 중복 쌍이 흐트러지지 않도록 유의하세요!  

구현한 함수를 활용하여 questions 와 answers 를 각각 que_corpus , ans_corpus 에 토큰화하여 저장합니다.  

In [53]:
from konlpy.tag import Mecab

mecab = Mecab()

In [54]:
questions = df_chatbotdata['Q'].values
answers = df_chatbotdata['A'].values

In [55]:
def build_corpus(src, tgt, tokenizer, max_len= 50) :
    
    temp_corpus = list(set(src + '\t' + tgt)) #중복 제거
    
    src_corpus, tgt_corpus = list(), list()
    
    for sen in temp_corpus :
        src_sen, tgt_sen = sen.split('\t')
        
        #src
        src_sen = preprocess_sentence(src_sen) # 전처리
        src_sen = tokenizer.morphs(src_sen) # 형태소 토크나이

        tgt_sen = preprocess_sentence(tgt_sen)
        tgt_sen = tokenizer.morphs(tgt_sen)
        
        if len(src_sen) < max_len and len(tgt_sen) < max_len :
            src_corpus.append(' '.join(src_sen))
            tgt_corpus.append(' '.join(tgt_sen))
        
    assert len(src_corpus) == len(tgt_corpus)   
    
    return src_corpus, tgt_corpus

In [56]:
# call build_corpus
corpus_questions, corpus_answers = build_corpus(questions, answers, mecab, max_len = 60)

In [57]:
assert len(corpus_questions) == len(corpus_answers)

print(len(corpus_questions))

11750


In [58]:
for i in range(5):
    print('Q:', corpus_questions[i])
    print('A:', corpus_answers[i])
    print(' ')

Q: 농구 하 다 무르팍 깨 짐
A: 약 바르 고 얼른 나 으세요 .
 
Q: 그녀 생각 만 하 면 멍 해져
A: 큐피드 의 화살 에 맞 았 나 봐요 .
 
Q: 도대체 왜 이러 는 걸까
A: 무슨 일 이 있 었 나 봐요 .
 
Q: 결혼 은 타이밍 인가 ?
A: 결혼 뿐 만 아니 라 인생 은 타이밍 의 연속 이 예요 .
 
Q: 그녀 를 다시 찾 았 습니다
A: 다행 이 네요 .
 


## Step 4. Augmentation
---
우리에게 주어진 데이터는 1만 개가량으로 적은 편에 속합니다. 이럴 때에 사용할 수 있는 테크닉을 배웠으니 활용해 봐야겠죠? Lexical Substitution을 실제로 적용해 보도록 하겠습니다.

아래 링크를 참고하여 한국어로 사전 훈련된 Embedding 모델을 다운로드합니다. Korean (w) 가 Word2Vec으로 학습한 모델이며 용량도 적당하므로 사이트에서 Korean (w)를 찾아 다운로드하고, ko.bin 파일을 얻으세요!

* [Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors)

다운로드한 모델을 활용해 데이터를 Augmentation 하세요! 앞서 정의한 lexical_sub() 함수를 참고하면 도움이 많이 될 겁니다.

Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록, 이후엔 반대로 원본 que_corpus 와 Augmentation된 ans_corpus 가 병렬을 이루도록 하여 전체 데이터가 원래의 3배가량으로 늘어나도록 합니다.


In [59]:
import os
from gensim.models import KeyedVectors, Word2Vec
from gensim.test.utils import datapath

model_path = os.getenv('HOME') + "/aiffel/transformer_chatbot/model/ko.bin"
model_path = './model/ko.bin'
wv = Word2Vec.load(model_path) # gensim 3.8.3 이하 버전에서만 동작함

In [60]:
wv.most_similar('영화') # similar words - korean

/var/folders/pc/h8hfx8zn0pb8vvgjpyjwq7cc0000gn/T/ipykernel_3275/1631014209.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  wv.most_similar('영화') # similar words - korean


[('다큐멘터리', 0.7265259027481079),
 ('영화사', 0.7152142524719238),
 ('드라마', 0.705294132232666),
 ('뮤지컬', 0.6947016716003418),
 ('코미디', 0.6909325122833252),
 ('영화인', 0.6702202558517456),
 ('서부극', 0.6571459174156189),
 ('스릴러', 0.6533164978027344),
 ('로맨스', 0.6428800225257874),
 ('애니메이션', 0.6425570249557495)]

### Lexical Substitution

In [61]:
def lexical_sub(sentence, word2vec):
    res = ""
    toks = mecab.morphs(sentence)

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]
        
    except:   # 단어장에 단어가 없으면 None을 리턴 
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res

In [62]:
sentence = '친구와 함께 영화를 보았다'
res = lexical_sub(sentence, wv)
print(res)

친구 와 함께 영화 을 보 았 다 


/var/folders/pc/h8hfx8zn0pb8vvgjpyjwq7cc0000gn/T/ipykernel_3275/1162967677.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  _to = word2vec.most_similar(_from)[0][0]


In [63]:
new_corpus = []

for old_question, old_answer in tqdm(zip(corpus_questions, corpus_answers)):
    new_question = lexical_sub(old_question, wv)
    new_answer = lexical_sub(old_answer, wv)
    if new_question is not None: 
        new_corpus.append(new_question +'\t'+ old_answer)
    if new_answer is not None: 
        new_corpus.append(old_question +'\t'+ new_answer)
    # Augmentation이 없더라도 원본 문장을 포함시킵니다
    new_corpus.append(old_question +'\t'+ old_answer)

0it [00:00, ?it/s]

/var/folders/pc/h8hfx8zn0pb8vvgjpyjwq7cc0000gn/T/ipykernel_3275/1162967677.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  _to = word2vec.most_similar(_from)[0][0]


In [64]:
print(len(new_corpus))
new_corpus = list(set(new_corpus))
print(len(new_corpus))

32122
32119


## Step 5. 데이터 벡터화
---
타겟 데이터인 ans_corpus 에 <start> 토큰과 <end> 토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화를 진행합니다. 우리가 구축한 ans_corpus 는 list 형태이기 때문에 아주 쉽게 이를 해결할 수 있답니다!

In [65]:
sample_data = ["12", "시", "땡", "!"]

print(["<start>"] + sample_data + ["<end>"])

['<start>', '12', '시', '땡', '!', '<end>']


위 소스를 참고하여 타겟 데이터 전체에 <start> 토큰과 <end> 토큰을 추가해 주세요!
챗봇 훈련 데이터의 가장 큰 특징 중 하나라고 하자면 바로 소스 데이터와 타겟 데이터가 같은 언어를 사용한다는 것이겠죠. 앞서 배운 것처럼 이는 Embedding 층을 공유했을 때 많은 이점을 얻을 수 있습니다.

특수 토큰을 더함으로써 ans_corpus 또한 완성이 되었으니, que_corpus 와 결합하여 전체 데이터에 대한 단어 사전을 구축하고 벡터화하여 enc_train 과 dec_train 을 얻으세요!

In [66]:
que_corpus = []
ans_corpus = []

for QnA in new_corpus : 
    question, answer = QnA.split('\t') #separate again into question and answer corpus
    que_corpus.append(question)
    ans_corpus.append(answer)

In [67]:
ans_corpus = ['<start> ' + answer + ' <end>' for answer in ans_corpus]
print(ans_corpus[0])

<start> 사랑 이 깊 네요 . <end>


### Train-Test Split

In [68]:
test_size = int(len(que_corpus) * 0.1)
print(test_size)

que_train = que_corpus[:-test_size]
ans_train = ans_corpus[:-test_size]

que_test = que_corpus[-test_size:]
ans_test = ans_corpus[-test_size:]

3211


### Tokenizer

In [69]:
# generate tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer.fit_on_texts(que_corpus + ans_corpus)

In [70]:
# creat seq tokens
enc_tensor = tokenizer.texts_to_sequences(que_train)
dec_tensor = tokenizer.texts_to_sequences(ans_train)

enc_tensor =  tf.keras.preprocessing.sequence.pad_sequences(enc_tensor, maxlen=52, padding='post')
dec_tensor =  tf.keras.preprocessing.sequence.pad_sequences(dec_tensor, maxlen=52, padding='post')

In [71]:
BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices((enc_tensor, dec_tensor)).batch(batch_size=BATCH_SIZE)

## Step 6. 훈련하기
---
앞서 번역 모델을 훈련하며 정의한 Transformer 를 그대로 사용하시면 됩니다! 대신 데이터의 크기가 작으니 하이퍼파라미터를 튜닝해야 과적합을 피할 수 있습니다. 모델을 훈련하고 아래 예문에 대한 답변을 생성하세요! 가장 멋진 답변과 모델의 하이퍼파라미터를 제출하시면 됩니다.

### 모델 구현

#### Positional Encoding

In [72]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

#### Mask

In [73]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask

#### MultiHeadAttention

In [74]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights

#### Position-wise Feed Forward Network

In [75]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out

#### Encoder Layer

In [76]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

#### Decoder Layer

In [77]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        '''
        Masked Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        '''
        Multi-Head Attention
        '''
        residual = out
        out = self.norm_2(out)
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

#### Encoder 클래스

In [78]:
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

#### Decoder 클래스

In [79]:
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

#### 트랜스포머 클래스

In [80]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

### 하이퍼 파라미터

In [81]:
# VOCAB_SIZE 
VOCAB_SIZE = len(tokenizer.index_word)
print(VOCAB_SIZE)

7963


In [82]:
d_model = 512

transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)

#### LR Scheduler

In [83]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

learning_rate = LearningRateScheduler(d_model)

### Optimizer

In [84]:
optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

### Loss Function

In [85]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

### Train Step

In [86]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

### Checkpoint

In [87]:
ckpt = tf.train.Checkpoint(step = tf.Variable(1), optimizer = optimizer , transformer = transformer)
manager = tf.train.CheckpointManager(ckpt, './checkpoints',max_to_keep=5)

### Train Model

In [88]:
def train_and_checkpoint(transformer, manager, EPOCH = 2):
    ckpt.restore(manager.latest_checkpoint)
    if manager.latest_checkpoint:
        print("Restored from {}".format(manager.latest_checkpoint))
    
    else:
        print("Initializing from scratch.")

    
    for epoch in range(EPOCH):
        ckpt.step.assign_add(1)
        total_loss = 0

        total_loss = 0

        dataset_count = tf.data.experimental.cardinality(train_dataset).numpy()
        tqdm_bar = tqdm(total=dataset_count)
        for step, (enc_batch, dec_batch) in enumerate(train_dataset):
            batch_loss, enc_attns, dec_attns, dec_enc_attns = \
            train_step(enc_batch,
                        dec_batch,
                        transformer,
                        optimizer)

            total_loss += batch_loss

            tqdm_bar.set_description_str('Epoch %2d' % (epoch + 1))
            tqdm_bar.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (step + 1)))
            tqdm_bar.update()
            
        if int(ckpt.step) % 2 == 0:
            save_path = manager.save()
            print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))

In [89]:
train_and_checkpoint(transformer, manager, EPOCH = 20)

Initializing from scratch.


  0%|          | 0/452 [00:00<?, ?it/s]

2023-02-10 08:48:21.905834: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-10 08:48:21.908957: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-10 08:50:35.600359: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Saved checkpoint for step 2: ./checkpoints/ckpt-1


  0%|          | 0/452 [00:00<?, ?it/s]

...

Saved checkpoint for step 18: ./checkpoints/ckpt-9


  0%|          | 0/452 [00:00<?, ?it/s]

  0%|          | 0/452 [00:00<?, ?it/s]

Saved checkpoint for step 20: ./checkpoints/ckpt-10


  0%|          | 0/452 [00:00<?, ?it/s]

###### 예문
1. 지루하다, 놀러가고 싶어.
2. 오늘 일찍 일어났더니 피곤하다.
3. 간만에 여자친구랑 데이트 하기로 했어.
4. 집에 있는다는 소리야.

---

###### 제출

Translations
> 1. 잠깐 쉬 어도 돼요 . <end>
> 2. 맛난 거 드세요 . <end>
> 3. 떨리 겠 죠 . <end>
> 4. 좋 아 하 면 그럴 수 있 어요 . <end>

Hyperparameters
> n_layers: 1
> d_model: 368
> n_heads: 8
> d_ff: 1024
> dropout: 0.2

Training Parameters
> Warmup Steps: 1000
> Batch Size: 64
> Epoch At: 10

## Step 7. 성능 측정하기
---
챗봇의 경우, 올바른 대답을 하는지가 중요한 평가 지표입니다. 올바른 답변을 하는지 눈으로 확인할 수 있겠지만, 많은 데이터의 경우는 모든 결과를 확인할 수 없을 것입니다. 주어진 질문에 적절한 답변을 하는지 확인하고, BLEU Score를 계산하는 calculate_bleu() 함수도 적용해 보세요.

In [91]:
def translate(tokens, model, src_tokenizer, tgt_tokenizer):
    tokens = mecab.morphs(tokens)
    tokens = tokenizer.texts_to_sequences(tokens)
    padded_tokens = tf.keras.preprocessing.sequence.pad_sequences(tokens,
                                                           maxlen=50,
                                                           padding='post')
    ids = []
    output = tf.expand_dims([tokenizer.word_index['<start>']], 0)   
    for i in range(50):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(padded_tokens, output)

        predictions, _, _, _ = model(padded_tokens, 
                                      output,
                                      enc_padding_mask,
                                      combined_mask,
                                      dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if tokenizer.word_index['<end>'] == predicted_id:
            result = ' '.join([tgt_tokenizer.index_word[i] for i in ids if i != 0])  
            return result

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = ' '.join([tgt_tokenizer.index_word[i] for i in ids if i != 0])  
    return result

print("슝=3")

슝=3


In [92]:
# examples :

examples = [
    '지루하다, 놀러가고 싶어.',
    '오늘 일찍 일어났더니 피곤하다.',
    '간만에 여자친구랑 데이트 하기로 했어.',
    '집에 있는다는 소리야.'
]

for i,example in enumerate(examples):
    
    print(f'quesions {i+1}:',example)
    print(f'reponse {i+1}:', translate(example, transformer, tokenizer, tokenizer))
    print()

quesions 1: 지루하다, 놀러가고 싶어.
reponse 1: 마음 이 가까운 곳 하 는 마음 을 가 보 세요 .

quesions 2: 오늘 일찍 일어났더니 피곤하다.
reponse 2: 오늘 도 즐거운 하루 보내 세요 .

quesions 3: 간만에 여자친구랑 데이트 하기로 했어.
reponse 3: 좋 은 날 이 죠 .

quesions 4: 집에 있는다는 소리야.
reponse 4: 마련 이 편하 겠 네요 .



### Beam Search

In [93]:
def beam_search_decoder(sentence, 
                        src_len,
                        tgt_len,
                        model,
                        src_tokenizer,
                        tgt_tokenizer,
                        beam_size):
    
    sentence = mecab.morphs(sentence)
    
    def calc_prob(src_ids, tgt_ids, model):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(src_ids, tgt_ids)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(src_ids, 
                tgt_ids,
                enc_padding_mask,
                combined_mask,
                dec_padding_mask)

        return tf.math.softmax(predictions, axis=-1)
    
    tokens = src_tokenizer.texts_to_sequences(sentence)
    
    src_in = tf.keras.preprocessing.sequence.pad_sequences(tokens,
                                                            maxlen=src_len,
                                                            padding='post')

    pred_cache = np.zeros((beam_size * beam_size, tgt_len), dtype=np.int64)
    pred_tmp = np.zeros((beam_size, tgt_len), dtype=np.int64)

    eos_flag = np.zeros((beam_size, ), dtype=np.int64)
    scores = np.ones((beam_size, ))

    pred_tmp[:, 0] = tgt_tokenizer.word_index['<start>']

    dec_in = tf.expand_dims(pred_tmp[0, :1], 0)
    prob = calc_prob(src_in, dec_in, model)[0, -1].numpy()

    for seq_pos in range(1, tgt_len):
        score_cache = np.ones((beam_size * beam_size, ))

        # init
        for branch_idx in range(beam_size):
            cache_pos = branch_idx*beam_size

            score_cache[cache_pos:cache_pos+beam_size] = scores[branch_idx]
            pred_cache[cache_pos:cache_pos+beam_size, :seq_pos] = \
            pred_tmp[branch_idx, :seq_pos]

        for branch_idx in range(beam_size):
            cache_pos = branch_idx*beam_size

            if seq_pos != 1:   # 모든 Branch를 로 시작하는 경우를 방지
                dec_in = pred_cache[branch_idx, :seq_pos]
                dec_in = tf.expand_dims(dec_in, 0)

                prob = calc_prob(src_in, dec_in, model)[0, -1].numpy()

            for beam_idx in range(beam_size):
                max_idx = np.argmax(prob)

                score_cache[cache_pos+beam_idx] *= prob[max_idx]
                pred_cache[cache_pos+beam_idx, seq_pos] = max_idx

                prob[max_idx] = -1

        for beam_idx in range(beam_size):
            if eos_flag[beam_idx] == -1: continue

            max_idx = np.argmax(score_cache)
            prediction = pred_cache[max_idx, :seq_pos+1]

            pred_tmp[beam_idx, :seq_pos+1] = prediction
            scores[beam_idx] = score_cache[max_idx]
            score_cache[max_idx] = -1

            if prediction[-1] == tgt_tokenizer.word_index['<end>']:
                eos_flag[beam_idx] = -1

    pred = []
    for long_pred in pred_tmp:
        zero_idx = long_pred.tolist().index(tgt_tokenizer.word_index['<end>'])
        short_pred = long_pred[:zero_idx+1]
        pred.append(short_pred)
        
    return  [' '.join([src_tokenizer.index_word[j] for j in i]) for i in pred]

In [94]:
MAX_LEN= 50

for i, example in enumerate(examples) :
    print(f'quesions {i+1}:',example)
    print(f'response {i+1}:', beam_search_decoder(example,
                        MAX_LEN,
                        MAX_LEN,
                        transformer,
                        tokenizer,
                        tokenizer,
                        beam_size=3))
    print()

quesions 1: 지루하다, 놀러가고 싶어.
response 1: ['<start> 마음 이 가까운 곳 하 는 마음 을 가 보 세요 . <end>', '<start> 마음 을 가까운 곳 하 는 마음 을 가 보 세요 . <end>', '<start> 마음 이 가까운 곳 하 나 마음 을 가 보 세요 . <end>']

quesions 2: 오늘 일찍 일어났더니 피곤하다.
response 2: ['<start> 오늘 도 즐거운 하루 보내 세요 . <end>', '<start> 오늘 도 즐거운 고생 보내 세요 . <end>', '<start> 오늘 도 즐거운 하루 보내 좋 . <end>']

quesions 3: 간만에 여자친구랑 데이트 하기로 했어.
response 3: ['<start> 좋 은 날 이 죠 . <end>', '<start> 고백 은 날 이 죠 . <end>', '<start> 인연 은 날 이 죠 . <end>']

quesions 4: 집에 있는다는 소리야.
response 4: ['<start> 마련 이 편하 겠 네요 . <end>', '<start> 마련 이 편하 겠 죠 . <end>', '<start> 같이 이 편하 겠 네요 . <end>']



### BLEU Single Score

In [95]:
def eval_bleu_single(model, src_sentence, tgt_sentence, src_tokenizer, tgt_tokenizer, verbose=True):
    src_tokens = tokenizer.texts_to_sequences(src_sentence)
    tgt_tokens = tokenizer.texts_to_sequences(tgt_sentence)

    if (len(src_tokens) > MAX_LEN): return None
    if (len(tgt_tokens) > MAX_LEN): return None

    reference = tgt_sentence.split()[1:-1]
    candidate = translate(src_sentence, model, src_tokenizer, tgt_tokenizer).split()

    score = sentence_bleu(reference, candidate,
                          smoothing_function=SmoothingFunction().method1)

    if verbose:
        print("Source Sentence: ", src_sentence)
        print("Model Prediction: ", candidate)
        print("Real: ", reference)
        print("Score: %lf\n" % score)
        
    return score
        
print('슝=3')

슝=3


In [99]:
# 인덱스를 바꿔가며 테스트해 보세요
test_idx = [0, 3, 7, 23, 59, 173]

for i in test_idx:
    eval_bleu_single(transformer, 
                 que_test[i], 
                 ans_test[i], 
                 tokenizer, 
                 tokenizer)

Source Sentence:  나 기재 됐 나 봐 
Model Prediction:  ['모두', '가', '사랑', '앞', '에', '어리석', '게', '아니', '라', '생각', '해요', '.']
Real:  ['확인', '해', '달', '라고', '해', '보', '세요', '.']
Score: 0.020256

Source Sentence:  공허 함 이 크 네
Model Prediction:  ['제', '놀드', '채워', '줄게요', '.']
Real:  ['저', '가', '채워', '줄게요', '.']
Score: 0.053728

Source Sentence:  일 하 는 곳 근처 가 서 염탐 하 고 왔 네 .
Model Prediction:  ['하루', '일', '수', '밖', '에', '없', '군요', '.']
Real:  ['그분', '에', '대일본', '궁금증', '을', '접', '어', '두', '셔요', '.']
Score: 0.036556

Source Sentence:  제 가 놓 았 던 손 을 다시 잡 으려니 힘드 네 .
Model Prediction:  ['마음', '이', '서로', '마음', '이', '안', '좋', '겠', '어요', '.']
Real:  ['원상', '복구', '는', '언제나', '힘드', '니까요', '.']
Score: 0.021105

Source Sentence:  이 나이 에 모르 는 게 왜 이렇게 많 지
Model Prediction:  ['나', '이', '아쉽', '네요', '.']
Real:  ['당연', '한', '것', '예요', '.']
Score: 0.053728

Source Sentence:  학교 에서 지속 잤 어 
Model Prediction:  ['선생', '님', '이', '요', '.']
Real:  ['한창', '잠', '이', '많', '을', '때', '죠', '.']
Score: 0.063894



### BLEU Score

In [97]:
def eval_bleu(model, src_sentences, tgt_sentence, src_tokenizer, tgt_tokenizer, verbose=True):
    total_score = 0.0
    sample_size = len(src_sentences)
    
    for idx in tqdm(range(sample_size)):
        score = eval_bleu_single(model, src_sentences[idx], tgt_sentence[idx], src_tokenizer, tgt_tokenizer, verbose)
        if not score: continue
        
        total_score += score
    
    print("Num of Sample:", sample_size)
    print("Total Score:", total_score / sample_size)
    
print("슝=3")

슝=3


In [ ]:
eval_bleu(transformer, que_test, ans_test, tokenizer, tokenizer, verbose=False)

  0%|          | 0/3211 [00:00<?, ?it/s]